In [ ]:
df = []
with open("vishnu.txt","r") as f:
  df.append(f.read())

In [ ]:
with open("phobia.txt","r") as f:
  df.append(f.read())

In [ ]:
df

['VISHNU SAHASRANAMA STOTRAM\nTAT1301 – (C.P.H.S – Elective Course) This course on Vishnu Sahasranama stotra, will prepare students to chant the slokas with clear pronunciation and furthermore, explore the meaning and essence of thousand names of Lord Vishnu expressed in the 108 slokas (verses) in the stotra.\n\nA late payment fee of $50 will be charged for all the Summer 2022 courses registrations after July 5, 2022.\nTotal course costs $250.\n\nCourse Code: TAT1301\nCredit Hours: 1\nCourse Level: 100\nDescription\nCourse Description:\nThis course on Vishnu Sahasranama stotra, will prepare students to chant the slokas with clear pronunciation. And furthermore, explore the meaning and essence of thousand names of Lord Vishnu expressed in the 108 slokas (verses) in the stotra. A brief background of Vishnu Sahasranama in the context of the ancient Mahabharata epic, will also be discussed with the introductory verses (Purva Bhaga) and the benefits of chanting Vishnu Shasranama in the conc

In [ ]:
!pip install -U sentence-transformers rank_bm25

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from sentence_transformers import util, SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm.autonotebook import tqdm
import numpy as np

In [ ]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 400
top_k = 32

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/383 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

Downloading:   0%|          | 0.00/794 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/316 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
corpus_embeddings = bi_encoder.encode(df, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#lowercase text and remove stop words
def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

tokenized_corpus = []
for passage in tqdm(df):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def search(query):
    #print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -2)[-2:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    #print("Top-3 lexical search (BM25) hits")
    #for hit in bm25_hits[0:3]:
    #    print("\t{:.3f}\t{}".format(hit['score'], df[hit['corpus_id']].replace("\n", " ")))

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, df[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    #print("\n-------------------------\n")
    #print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    #for hit in hits[0:3]:
    #    print("\t{:.3f}\t{}".format(hit['score'], df[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    #print("\n-------------------------\n")
    #print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    top_ranks = ""
    for hit in hits[0:2]:
        #print("\t{:.3f}\t{}".format(hit['cross-score'], df[hit['corpus_id']].replace("\n", " ")))
        #top_ranks+=df[hit['corpus_id']]
        return df[hits[0]['corpus_id']]


In [ ]:
search("what is the class structure for hinduphobia course?")

'Course Name: Understanding Hinduphobia . Note: A late payment fee of $50 will be charged for Summer quarter registrations submitted after July 5, 2022. Course Fee: $350.00. Course Code: HSF2201. Credit Hours: 1.5. Course Level: 100. . Description. Course Description:. The term Hinduphobia has been in circulation since the late nineteenth century and the phenomenon has been experienced and discussed in Hindu community spaces and literature for generations. However, the formal academic study of Hinduphobia remains sparse, in no small part due to Western academic consensus that Hinduism is foundationally flawed and suspect. This gap between community experience and knowledge and academic discourse about Hinduism sits at the heart of Hinduphobia. Recently, with the development of the Working Definition of Hinduphobia* pathways for a more methodical academic study have come clearly into view. The definition offers an operationalization of the phenomenon, creating a supporting framework for

# Two courses

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163 kB 9.6 MB/s 
  Created wheel for openai: filename=openai-0.20.0-py3-none-any.whl size=54118 sha256=2eb1f537cff1b2baa84b63ec7c41a7b95de6821c51e7109a1a5da0f27b6baf2f
  Stored in directory: /root/.cache/pip/wheels/71/8d/9b/e28529ec53123e0279208f99148d4661232120d78cb866839b
Successfully built openai


In [ ]:
import openai
import json
with open('GPT_SECRET_KEY.json') as f:
    data = json.load(f)
openai.api_key = data["API_KEY"]

In [ ]:
def ask_question(question):
    s = search(question)
    text = "This is a bot that answers questions from the paragraph given below: \n"
    text+=s;
    text+="\nThe question from the paragraph is "
    text+=question+"\n"
    #print(text+"\n")
    final = openai.Completion.create(
          model="text-davinci-002",
          prompt=text,
          temperature=0,
          max_tokens=300,
          frequency_penalty=0.5,
          presence_penalty=0
        )
    print(final["choices"][0]["text"])
    #return final["choices"][0]["text"]

In [ ]:
ask_question("who is the instructor for VISHNU SAHASRANAMA STOTRAM course?")


The instructor for VISHNU SAHASRANAMA STOTRAM course is Acharya Mrityunjay Mani Trivedi.


In [ ]:
ask_question("is there a project for hinduphobia course?")


Yes, there is a project for hinduphobia course.


# Single course

In [ ]:
with open("story.txt","r") as f:
  story = f.read()

In [ ]:
elements = []
separator = "Course content:"
elements.append(f"{separator}: {story.split(separator)[1]}")
elements.append(story.split(separator)[0])

In [ ]:
elements[1]

In [ ]:
separator = "Course Learning Objectives:"
elements.append(f"{separator}: {elements[0].split(separator)[1]}")
elements[0] = elements[0].split(separator)[0]

In [ ]:
separator = "Class Structure"
elements.append(f"{separator}: {elements[2].split(separator)[1]}")
elements[2] = elements[2].split(separator)[0]

In [ ]:
separator = "Area of Study"
elements.append(f"{separator}: {elements[3].split(separator)[1]}")
elements[3] = elements[3].split(separator)[0]

In [ ]:
elements

In [ ]:
bi_encoder_story = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder_story.max_seq_length = 500
top_k_story = 4

In [ ]:
cross_encoder_story = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [ ]:
corpus_embeddings_story = bi_encoder_story.encode(elements, convert_to_tensor=True, show_progress_bar=True)

In [ ]:
def search_story(query):
    #print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -2)[-2:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    #print("Top-3 lexical search (BM25) hits")
    #for hit in bm25_hits[0:3]:
    #    print("\t{:.3f}\t{}".format(hit['score'], elements[hit['corpus_id']].replace("\n", " ")))

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder_story.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings_story, top_k=top_k_story)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, elements[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder_story.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    #print("\n-------------------------\n")
    #print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    #for hit in hits[0:3]:
    #    print("\t{:.3f}\t{}".format(hit['score'], elements[hit['corpus_id']].replace("\n", " ")))

    # Output of top-5 hits from re-ranker
    #print("\n-------------------------\n")
    #print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    top_ranks = ""
    for hit in hits[0:2]:
        print("\t{:.3f}\t{}".format(hit['cross-score'], elements[hit['corpus_id']].replace("\n", " ")))
        top_ranks+=elements[hit['corpus_id']]
    return top_ranks


In [ ]:
def ask_question_story(question):
    s = search_story(question)
    text = "This is a bot that answers questions from the paragraph given below: \n"
    text+=s;
    text+="\nThe question from the paragraph is "
    text+=question+"\n"
    #print(text+"\n")
    final = openai.Completion.create(
          model="text-davinci-002",
          prompt=text,
          temperature=0,
          max_tokens=300,
          frequency_penalty=0.5,
          presence_penalty=0
        )
    print(final["choices"][0]["text"])
    #return final["choices"][0]["text"]